In [1]:
import re

def is_well_formed(expression):
    """
    Check if the expression is a well-formed sentential logic expression.
    """
    expression = expression.replace(" ", "").replace("\t", "")  # Remove spaces/tabs

    if not expression:
        return False, None

    # Ensure expression contains at least one atomic proposition
    if not re.search(r'[A-Z]', expression):
        return False, None

    # Step 1: Check for balanced parentheses
    if not is_balanced(expression):
        return False, None

    # Step 2: Check for invalid operators and malformed patterns
    if has_invalid_operators(expression):
        return False, None

    # Step 3: Extract the main connective
    main_connective = get_main_connective(expression)

    return (True, main_connective) if main_connective else (False, None)

def is_balanced(expression):
    """
    Check if parentheses are balanced efficiently.
    """
    stack = 0
    for char in expression:
        if char == '(':
            stack += 1
        elif char == ')':
            if stack == 0:
                return False
            stack -= 1
    return stack == 0

def has_invalid_operators(expression):
    """
    Check for malformed operators like duplicated, misplaced, or invalid patterns.
    """
    invalid_patterns = [
        r'\(\)',  # Empty parentheses
        r'∧∧', r'∨∨', r'→→',  # Duplicated operators
        r'~~(?![A-Z(~])',  # Corrected: Allows negations like ~~~P # Double negation must be followed by a valid term
        r'~[∧∨→]', r'[∧∨→]~',  # Misplaced negation
        r'^[∧∨→]', r'[∧∨→]$',  # Operators at the start or end
        r'\)\(',  # Misplaced parentheses
        r'(?<![A-Z)])→', r'→(?![A-Z(])',  # Binary connectives must have valid operands
        r'(?<![∧∨→])\(\)[∧∨→]',  # Operators inside parentheses without operands
    ]
    print(f"Checking expression: {expression}")
    for pattern in invalid_patterns:
        if re.search(pattern, expression):
            print(f"Triggered invalid pattern: {pattern}")

    return any(re.search(pattern, expression) for pattern in invalid_patterns)

def get_main_connective(expression):
    """
    Extract the main connective of a well-formed sentential logic expression.
    """
    expression = expression.strip()
    original_expression = expression  # Save the original expression for comparison

    # Remove redundant outer parentheses while ensuring correctness
    while expression.startswith('(') and expression.endswith(')') and is_balanced(expression[1:-1]):
        expression = expression[1:-1]

    # Case 1: Atomic sentence (must not be enclosed in parentheses)
    if len(expression) == 1 and expression.isalpha() and expression.isupper():
        # If the original expression had parentheses, it's invalid
        if original_expression != expression:
            return None
        return expression

    # Case 2: Negation (ensure valid operand follows)
    if expression.startswith('~'):
        remainder = expression[1:]
        # Recursively check if the remainder is a valid expression
        if is_well_formed(remainder)[0]:  # Check if the remainder is well-formed
            return '~'

    # Case 3: Binary connectives
    stack = 0
    main_connective = None
    lowest_precedence = float('inf')
    precedence = {'→': 1, '∨': 2, '∧': 3}  # Lower number = lower precedence

    for i, char in enumerate(expression):
        if char == '(':
            stack += 1
        elif char == ')':
            stack -= 1
        elif stack == 0 and char in precedence:  # Only consider top-level operators
            # Ensure both left and right operands exist
            if i > 0 and i < len(expression) - 1 and expression[i-1] not in "(∧∨→" and expression[i+1] not in ")∧∨→":
                if precedence[char] < lowest_precedence:
                    lowest_precedence = precedence[char]
                    main_connective = char

    return main_connective

def process_file(input_file, output_file):
    """
    Process the input file and write the results to the output file.
    """
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        for line in infile:
            line = line.strip()
            if not line:
                continue

            well_formed, result = is_well_formed(line)
            outfile.write(result if well_formed else '∅')
            outfile.write('\n')  # Add line breaks for readability

# File names for usage
input_file = 'input_Testing.txt'
output_file = 'output_Testing.txt'
process_file(input_file, output_file)

Checking expression: P
Checking expression: Q
Checking expression: R
Checking expression: P
Checking expression: P∧Q
Checking expression: P∧Q
Checking expression: (P∧Q)
Checking expression: (P∧Q)
Checking expression: P∧(Q∨R)∧S
Checking expression: P∧(Q∨R→S)
Checking expression: P∧(Q∨R→S)∧T
Checking expression: P∧(Q∨R→S)∧(T∨U)
Checking expression: (P∧(Q∨R))
Checking expression: (P∧(Q→(R∨S)))
Checking expression: ((((P∧Q)→R)∨S)∧T)
Checking expression: ((P∨Q)∧(R∨S))
Checking expression: (Q∨R)
Checking expression: (P∧Q)∨R
Checking expression: (P∨Q)
Checking expression: P→(Q∧R)
Checking expression: P∧(Q∨R→S)∧(T∨U)→V
Checking expression: (P→Q)
Checking expression: (P→(Q→R))
Checking expression: ((P∧Q)→(R∨S))
Checking expression: (P→(Q→(R→S)))
Checking expression: (((P∧Q)∨R)→S)
Checking expression: P→Q→R
Checking expression: ~(P∨Q)
Checking expression: (P∨Q)
Checking expression: ~P
Checking expression: P
Checking expression: ~~P
Checking expression: ~P
Checking expression: P
Checking expressi